# Django Tutorial (part 4)
Back to the project home:

In [1]:
cd myworkspace/mysite
tree -I __pycache__

.
├── db.sqlite3
├── manage.py
├── mysite
│   ├── asgi.py
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── polls
    ├── admin.py
    ├── apps.py
    ├── __init__.py
    ├── migrations
    │   ├── 0001_initial.py
    │   └── __init__.py
    ├── models.py
    ├── tests.py
    ├── urls.py
    └── views.py

3 directories, 16 files


Restart the server:

In [2]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

[1] 66093


Add some more views:

In [3]:
cat << EOF > polls/views.py
from django.http import HttpResponse

def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")
    
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)
    
EOF

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
^JOctober 07, 2021 - 15:57:29
Django version 3.2.8, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.


Wire these new views into the polls.urls module by adding the following path() call

In [4]:
cat << EOF > polls/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # ex: /polls/
    path('', views.index, name='index'),
    # ex: /polls/5/
    path('<int:question_id>/', views.detail, name='detail'),
    # ex: /polls/5/results/
    path('<int:question_id>/results/', views.results, name='results'),
    # ex: /polls/5/vote/
    path('<int:question_id>/vote/', views.vote, name='vote'),
]
EOF

/home/chris/home/workspace/python-course/src/37 Django/myworkspace/mysite/polls/views.py changed, reloading.


Take a look in your browser of some url concerning question 34 (for example) – these will display the placeholder results and voting pages.

In [5]:
firefox http://localhost:7000/polls
sleep 5
firefox http://localhost:7000/polls/34/results
sleep 5
firefox http://localhost:7000/polls/34/vote

Watching for file changes with StatReloader
Performing system checks...

firefox http://localhost:7000/polls^JSystem check identified no issues (0 silenced).
October 07, 2021 - 15:57:30
Django version 3.2.8, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.
[07/Oct/2021 15:57:30] "GET /polls/ HTTP/1.1" 200 40
[07/Oct/2021 15:57:35] "GET /polls/34/results/ HTTP/1.1" 200 45


: 1

Add some views that do something:

In [6]:
pwd
cat << EOF >polls/views.py
from django.http import HttpResponse
from .models import Question
    
def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    output = ', '.join([q.question_text for q in latest_question_list])
    return HttpResponse(output)
    
def detail(request, question_id):
    return HttpResponse(f"You're looking at question {question_id}.")

def results(request, question_id):
    response = f"You're looking at the results of question {question_id}."
    return HttpResponse(response)

def vote(request, question_id):
    return HttpResponse(f"You're voting on question {question_id}")    

EOF

/home/chris/home/workspace/python-course/Level 4/05 Django/myworkspace/mysite


check out the urls in the browser:

In [7]:
firefox http://localhost:7000/polls
sleep 5
firefox http://localhost:7000/polls/34/results
sleep 5
firefox http://localhost:7000/polls/34/vote

/home/chris/home/workspace/python-course/src/37 Django/myworkspace/mysite/polls/views.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
October 07, 2021 - 15:58:20
Django version 3.2.8, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.
firefox http://localhost:7000/polls^J[07/Oct/2021 15:58:21] "GET /polls/ HTTP/1.1" 200 11
[07/Oct/2021 15:58:26] "GET /polls/34/results/ HTTP/1.1" 200 45
[07/Oct/2021 15:58:32] "GET /polls/34/vote/ HTTP/1.1" 200 28


That works, but hard coding is bad.  The alternative is to use templates:

In [ ]:
mkdir -p polls/templates/polls

Now create the template - note we create the template in the directory `polls/templates/polls`:

In [ ]:
cat << EOF > polls/templates/polls/index.html
{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}
EOF

Modify the view to link to the template.

In [ ]:
cat << EOF >polls/views.py
from django.http import HttpResponse
from django.template import loader

from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html')
    context = {
        'latest_question_list': latest_question_list,
    }
    return HttpResponse(template.render(context, request))
    
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)    
EOF

Check out the app.

In [ ]:
firefox http://localhost:7000/polls

It’s a very common idiom to load a template, fill a context and return an HttpResponse object with the result of the rendered template. Django provides a shortcut with `render`. 

Here’s the full index() view, rewritten:

In [ ]:
cat << EOF >polls/views.py
from django.shortcuts import render
from django.http import HttpResponse
from django.template import loader
from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}
    return render(request, 'polls/index.html', context)
    
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)    
EOF

Check it still works:

In [ ]:
firefox http://localhost:7000/polls

Add some error handling by modifying `detail()`:

In [ ]:
cat << EOF >polls/views.py
from django.http import Http404
from django.http import HttpResponse
from django.shortcuts import render
from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}
    return render(request, 'polls/index.html', context)
    
def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, 'polls/detail.html', {'question': question})
    
def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)    
EOF

Now add a template that will generate a 404: Page not found" error

In [ ]:
cat << EOF >polls/templates/polls/detail.html
{{ question }}
EOF

and fire up `firefox`

In [ ]:
firefox http://localhost:7000/polls/detail/34

Add a more realistic template:

In [ ]:
cat << EOF >polls/templates/polls/detail.html
<h2>This is template 'polls/detail.html'</h2>
<h2>{{ question.question_text }}</h2>
<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }}</li>
{% endfor %}
</ul>
EOF

Try it out:

In [ ]:
firefox http://localhost:7000/polls/

Finally, let's take a look at our project structure:

In [ ]:
tree -I __pycache__

You can find further details on the original tutorial:  
https://docs.djangoproject.com/en/3.2/intro/tutorial01/